In [1]:
!pip install pytest ipytest pytest-csv pytest-benchmark

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [2]:
import numpy as np
import pytest
%matplotlib inline
from coffea import hist
import coffea.processor as processor
import awkward as ak

from dask.distributed import Client, LocalCluster
import time
import os
import ipytest

ipytest.config(rewrite_asserts=True, magics=True)

{'rewrite_asserts': True,
 'magics': True,
 'clean': '[Tt]est*',
 'addopts': (),
 'run_in_thread': False,
 'defopts': True}

In [3]:
fileset = {'SingleMu' : ["root://eospublic.cern.ch//eos/root-eos/benchmark/Run2012B_SingleMu.root"]}

from dask.distributed import Client, Worker, WorkerPlugin
from typing import List
import os

class DependencyInstaller(WorkerPlugin):
    def __init__(self, dependencies: List[str]):
        self._depencendies = " ".join(f"'{dep}'" for dep in dependencies)

    def setup(self, worker: Worker):
        os.system(f"pip install {self._depencendies}")


dependency_installer = DependencyInstaller([
    "pytest-benchmark",
])

client = Client("tls://localhost:8786")
#Uncomment only if we would like to compare the same number of workers
#cluster = CoffeaCasaCluster()
#cluster.scale(10)
#client = Client(cluster)
client.register_worker_plugin(dependency_installer)

OSError: Timed out trying to connect to tls://red-c5119.unl.edu:49180 after 10 s

In [4]:
# This program plots an event-level variable (in this case, MET, but switching it is as easy as a dict-key change). It also demonstrates an easy use of the book-keeping cutflow tool, to keep track of the number of events processed.

# The processor class bundles our data analysis together while giving us some helpful tools.  It also leaves looping and chunks to the framework instead of us.
class Processor(processor.ProcessorABC):
    def __init__(self):
        # Bins and categories for the histogram are defined here. For format, see https://coffeateam.github.io/coffea/stubs/coffea.hist.hist_tools.Hist.html && https://coffeateam.github.io/coffea/stubs/coffea.hist.hist_tools.Bin.html
        dataset_axis = hist.Cat("dataset", "")
        muon_axis = hist.Bin("Jet_pt", "Jet_pt [GeV]", 100, 15, 200)
        
        # The accumulator keeps our data chunks together for histogramming. It also gives us cutflow, which can be used to keep track of data.
        self._accumulator = processor.dict_accumulator({
            'Jet_pt': hist.Hist("Counts", dataset_axis, muon_axis),
            'cutflow': processor.defaultdict_accumulator(int)
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, events):
        output = self.accumulator.identity()
        
        # This is where we do our actual analysis. The dataset has columns similar to the TTree's; events.columns can tell you them, or events.[object].columns for deeper depth.
        dataset = events.metadata["dataset"]
        
        muons = events.Muon
        electrons = events.Electron
        jets = events.Jet
        
        # We can define a new key for cutflow (in this case 'all events'). Then we can put values into it. We need += because it's per-chunk (demonstrated below)
        output['cutflow']['all events'] += ak.size(jets, axis=0)
        output['cutflow']['all jets'] += ak.sum(ak.num(jets, axis=1))
        
        # This fills our histogram once our data is collected. The hist key ('MET=') will be defined in the bin in __init__.
         # Get jets with higher GeV than 30.
        min_jetpt = (jets.pt > 30)
        output['cutflow']['jets with pt > 30'] += ak.sum(ak.sum(min_jetpt, axis=1))
        
        # Get all leptons with higher GeV than 10.
        min_muonpt = (muons.pt > 10)
        output['cutflow']['muons with pt > 10'] += ak.sum(ak.sum(min_muonpt, axis=1))
        min_electronpt = (electrons.pt > 10)
        output['cutflow']['electrons with pt > 10'] += ak.sum(ak.sum(min_electronpt, axis=1))
        
        # Mask jets and leptons with their minimum requirements/
        goodjets = jets[min_jetpt]
        goodmuons = muons[min_muonpt]
        goodelectrons = electrons[min_electronpt]
    
        jet_muon_pairs = ak.cartesian({'jets': goodjets, 'muons': goodmuons}, nested=True)
        jet_electron_pairs = ak.cartesian({'jets': goodjets, 'electrons': goodelectrons}, nested=True)
    
        # This long conditional checks that the jet is at least 0.4 euclidean distance from each lepton. It then checks if each unique jet contains a False, i.e., that a jet is 0.4 euclidean distance from EVERY specific lepton in the event.
        good_jm_pairs = goodjets.nearest(goodmuons).delta_r(goodjets) > 0.4
        good_je_pairs = goodjets.nearest(goodelectrons).delta_r(goodjets) > 0.4
        good_jl_pairs = good_jm_pairs & good_je_pairs
        
        output['cutflow']['jet-muon pairs'] += ak.sum(ak.sum(good_jm_pairs, axis=1))
        output['cutflow']['jet-electron pairs'] += ak.sum(ak.sum(good_je_pairs, axis=1))
        output['cutflow']['jet-lepton pairs'] += ak.sum(ak.sum(good_jl_pairs, axis=1))
        
        # We then mask our jets with all three of the above good pairs to get only jets that are 0.4 distance from every type of lepton, and sum them.
        sumjets = ak.sum(goodjets[good_jl_pairs].pt, axis=1)
        output['cutflow']['final jets'] += ak.sum(ak.num(goodjets[good_jl_pairs], axis=1))
        output['Jet_pt'].fill(dataset=dataset, Jet_pt=sumjets)
        
        return output


    def postprocess(self, accumulator):
        return accumulator

In [5]:
# Function which we are interested to benchmark where chunk_size is changed dependending on iteration of benchmark run.
def coffea_processor_adlexample7(chunk_size):
  output = processor.run_uproot_job(fileset,
                                  treename = 'Events',
                                  processor_instance = Processor(),
                                  executor = processor.dask_executor,
                                  chunksize = chunk_size,
                                  executor_args = {'schema': processor.NanoAODSchema,
                                                   'client': client,
                                                   'savemetrics': True}
                                )
  return output

In [6]:
@pytest.mark.parametrize("chunk_size", range(100000, 200000, 100000))
def test_coffea_processor_adlexample7(benchmark, chunk_size):
        output = benchmark(coffea_processor_adlexample7, chunk_size)
        # Custom metrics available with `savemetrics` option
        benchmark.extra_info['events_s_thread'] = output[1]['entries'] / output[1]['processtime']

In [ ]:
ipytest.run("-qq")